## DE300 - Homework 3
### Aden Benson
#### Part 1 - tf-idf

For the following code block, AI was used behind the scenes in the root shell of my docker container to install Java. I could either have done this by editing the dockerfile and rebuilding the image, or going into the shell and running some Java-related commands to install and link to the right location. I chose to do the second option.

In [1]:
from pyspark.sql.functions import split, col, explode, lower, length

from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .master("local[*]")
         .appName("AG news")
         .getOrCreate()
        )

sc = spark.sparkContext

agnews = spark.read.csv("dataset/agnews_clean.csv", inferSchema=True, header=True)

# turning the second column from a string to an array
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType
agnews = agnews.withColumn('filtered', F.from_json('filtered', ArrayType(StringType())))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/26 19:39:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# each row contains the document id and a list of filtered words
agnews.show(5, truncate=30)

+---+------------------------------+
|_c0|                      filtered|
+---+------------------------------+
|  0|[wall, st, bears, claw, bac...|
|  1|[carlyle, looks, toward, co...|
|  2|[oil, economy, cloud, stock...|
|  3|[iraq, halts, oil, exports,...|
|  4|[oil, prices, soar, time, r...|
+---+------------------------------+
only showing top 5 rows



25/05/26 19:39:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


Task 1 - Design the MapReduce function for calculating tf-idf measure.

In [3]:
# Create a pyspark df that contains each word and its associated document. This is what we'll perform mapreduce on.
words=agnews.select(col('_c0'), explode(col('filtered')).alias('word'))
words.show()

+---+---------+
|_c0|     word|
+---+---------+
|  0|     wall|
|  0|       st|
|  0|    bears|
|  0|     claw|
|  0|     back|
|  0|    black|
|  0|  reuters|
|  0|  reuters|
|  0|    short|
|  0|  sellers|
|  0|     wall|
|  0|   street|
|  0|dwindling|
|  0|     band|
|  0|    ultra|
|  0|   cynics|
|  0|   seeing|
|  0|    green|
|  1|  carlyle|
|  1|    looks|
+---+---------+
only showing top 20 rows



25/05/26 19:39:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


First, design mapreduce functions for finding "For each $d$, the counts of $t$"

In [4]:
# Design the map phase, which reads a row and returns the key of (word, doc number) and a value of 1
def map_phase_1(row):
    dict=row.asDict()
    doc_num=dict['_c0']
    word_str=dict['word']

    # Return a tuple of ((doc_num, word), 1)
    return ((doc_num, word_str), 1.0)

In [5]:
# Design the sort/reduce phase, which adds up all the same instances of (doc_num, word) to find the counts.
doc_word_counts=(words
        .rdd
        .map(map_phase_1)
        .reduceByKey(lambda a,b: a+b)
       )

The above code outputs text files that have keys of document number and word, along with the count of that word in the document. This gives us "For each $d$, the counts of $t$"

Next, design Mapreduce to create an rdd with the number of words in each doc.

In [6]:
# Using words, create key value pairs of (doc_number, 1) for each word that appears.
def map_phase_2(row):
    dict=row.asDict()
    doc_num=dict['_c0']
    return (doc_num, 1.0)

In [7]:
# Design a reduce function that finds number of words per document
num_words_per_doc=(words
                   .rdd
                   .map(map_phase_2)
                   .reduceByKey(lambda a,b: a+b)
)

In [8]:
# This is a much smaller rdd, so we can show that it successfully counts the number of words in each document.
num_words_per_doc.take(5)

25/05/26 19:39:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


[(0, 18.0), (9, 19.0), (18, 21.0), (27, 21.0), (36, 26.0)]

Finally, use Mapreduce to find the counts of $d$ that contain $t$ for each $t$

In [9]:
# First, create a Words df that only contains unique values for each document
# This gets rid of entries where documents have duplicate word entries
unique_words=words.select(col('_c0'), col('word')).distinct()

In [10]:
# Design map phase, which returns (word, 1) each time a word appears in a row
def map_phase_3(row):
    dict=row.asDict()
    word=dict['word']
    return (word, 1.0)

In [11]:
# Design reduce phase, which finds the count that each word appears in a document
word_counts=(
    unique_words
    .rdd
    .map(map_phase_3)
    .reduceByKey(lambda a,b: a+b)
)

25/05/26 19:39:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


In [12]:
word_counts.take(5)

[('nation', 1598.0),
 ('eggs', 38.0),
 ('software', 3798.0),
 ('different', 410.0),
 ('7', 2614.0)]

Task 2 - Calculate tf-idf measure for each row in the agnews_clean.csv. Save the measures in a new column.

Let $t$ be a term (a word), $d$ be a document, and $D$ be the collection of the documents.

Term frequency (tf):

$$\mathrm{tf}(t, d) = \frac{\textrm{\# occurrences of } t \textrm{ in } d}{\textrm{\# terms in } d},$$

Inverse document frequency (idf): $$\mathrm{idf}(t, D) = \log\left(\frac{\textrm{\# docs in } D}{\textrm{\# docs containing } t}\right).$$

As a result, the tf-idf measure is

$$\textrm{tf-idf}(t, d, D) = \mathrm{tf}(t, d)\times \mathrm{idf}(t, D).$$

In [13]:
### First, calculate tf(t,d)

# Add # terms in d to doc_word_counts, which already contains # occurrences of t in d

# Start by changing format of doc_word_counts from ((doc, word), count) to (doc, (word, count)) so we can join along document key
def transform_helper(row):
    return (row[0][0], (row[0][1], row[1]))
doc_word_counts_tf=doc_word_counts.map(transform_helper)

# Join doc_word_counts along doc number with num_words_per_doc
joined_tf=doc_word_counts_tf.join(num_words_per_doc)

# Compute tf(t,d) term
def tf_t_d(row):
    tf_t_d_calc=row[1][0][1]/row[1][1]
    return ((row[0], row[1][0][0]), tf_t_d_calc)
computed_tf=joined_tf.map(tf_t_d)


In [14]:
# Now, we have computed_tf which shows #d#, #t#, and the tf(t,d) measure
computed_tf.take(1)

25/05/26 19:40:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , filtered
 Schema: _c0, filtered
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


[((0, 'st'), 0.05555555555555555)]

In [15]:
### Next, calculate idf(t,d)

# Find number of docs in D
num_docs=agnews.select(col('_c0')).distinct().count()

# Add the idf(t,D) metric to each row of word_counts
import numpy as np
def idf_t_D(row):
    idf_t_D_calc=np.log(num_docs/row[1])
    return (row[0], idf_t_D_calc)
computed_idf=word_counts.map(idf_t_D)

25/05/26 19:40:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 
 Schema: _c0
Expected: _c0 but found: 
CSV file: file:///Users/adenbenson/Desktop/spring_25/Data_Eng%20300/homeworks/hw3/dataset/agnews_clean.csv


In [16]:
### Finally, compute tf-idf measure

# Transform computed_tf so the key is only on the word
def tf_transform_helper(row):
    return (row[0][1], (row[0][0], row[1]))
transformed_computed_tf=computed_tf.map(tf_transform_helper)

# Add idf measure to tf RDD
full_rdd=transformed_computed_tf.join(computed_idf)

# Compute tf-idf measure
def compute_tf_idf(row):
    tf_idf_calc=row[1][0][1]*row[1][1]
    return ((row[1][0][0], row[0]), float(tf_idf_calc))
tf_idf_rdd=full_rdd.map(compute_tf_idf)


In [17]:
# Transform our RDD back into a pyspark DF to display our outputs

# First, use map to remove nested tuples. Then, use .toDF to turn into a Pyspark Dataframe
tf_idf_df=tf_idf_rdd.map(lambda i: (i[0][0], i[0][1], i[1])).toDF(['doc_Number', 'word', 'tf_idf_measure'])

tf_idf_df.show(5)

/Users/adenbenson/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


+----------+-------+-------------------+
|doc_Number|   word|     tf_idf_measure|
+----------+-------+-------------------+
|        18|deficit|  0.540640233790213|
|     11970|deficit|0.24681401977379286|
|     20826|deficit|0.23653010228321816|
|     21078|deficit|0.23170295733866267|
|     37116|deficit|0.24681401977379286|
+----------+-------+-------------------+
only showing top 5 rows



Task 3 - Print out the tf-idf measure for the first 5 documents.

In [18]:
# Sort our df by doc_Number, then print out all words in the first 5 documents with the tf-idf measure
tf_idf_df_sorted=tf_idf_df.orderBy(col('doc_Number'))

# Use a for loop to print out each df by doc_Number, since one table for all words was too large
for i in range(5):
    tf_idf_df_filtered=tf_idf_df_sorted.where(col('doc_Number')==i)
    tf_idf_df_filtered.show()

+----------+---------+-------------------+
|doc_Number|     word|     tf_idf_measure|
+----------+---------+-------------------+
|         0|   seeing|0.37743394553516213|
|         0|    short| 0.2773120373951269|
|         0|     claw|  0.499114829314058|
|         0|     back| 0.1892216338539946|
|         0|    green| 0.2877107940095433|
|         0|dwindling| 0.4572386180709258|
|         0|    ultra| 0.4125512394225831|
|         0|       st| 0.2584728642725166|
|         0|  sellers| 0.4468379768438066|
|         0|   street|0.24678348986493034|
|         0|   cynics|  0.563734318747707|
|         0|    bears| 0.3372044607529448|
|         0|     wall| 0.5115985326511431|
|         0|  reuters|0.24754017186645658|
|         0|    black| 0.2953171727366614|
|         0|     band| 0.3643421454792778|
+----------+---------+-------------------+



+----------+----------+-------------------+
|doc_Number|      word|     tf_idf_measure|
+----------+----------+-------------------+
|         1|    placed| 0.2284965552404658|
|         1|  industry|0.15043731768548949|
|         1| aerospace| 0.2581171817448437|
|         1|   quietly|0.25188254045524316|
|         1|     looks| 0.1973537176743789|
|         1|      bets|0.27861293130724324|
|         1|    toward| 0.1898997183872362|
|         1|   carlyle| 0.7168306746824437|
|         1|      firm|0.15969712503706046|
|         1|   defense| 0.1751279339938823|
|         1|     plays|0.22418048797172685|
|         1|investment| 0.1890771769001148|
|         1|commercial| 0.2057832028092643|
|         1|    market|0.13394932212703356|
|         1|   private| 0.1929050573011279|
|         1|reputation| 0.2578098186776328|
|         1|     timed|  0.324478643568105|
|         1|    making| 0.1698717076460444|
|         1|     group|0.12468100563149095|
|         1|      part|0.1602203

+----------+--------+-------------------+
|doc_Number|    word|     tf_idf_measure|
+----------+--------+-------------------+
|         2|expected|0.16094627131903613|
|         2|earnings| 0.1792714404894228|
|         2|   stock|0.17879168082328206|
|         2|  prices|0.14472559202114177|
|         2|    hang|0.30475018305843793|
|         2| worries|0.23009353850726894|
|         2|  stocks|0.14976769101715193|
|         2|   depth|0.31343954772064864|
|         2| outlook| 0.4265073217271922|
|         2|     oil|0.13908157105107033|
|         2|  summer|0.22694739048609625|
|         2|  market|0.15069298739291276|
|         2|    next|0.14062721303262238|
|         2| soaring| 0.2596334462817101|
|         2|   crude|  0.197241148492091|
|         2| economy| 0.3721400726458204|
|         2|    week|0.13121900794126834|
|         2|    plus|0.24449073714833106|
|         2|   cloud|  0.295159450642955|
|         2| reuters|0.18565512889984243|
+----------+--------+-------------

+----------+--------------+-------------------+
|doc_Number|          word|     tf_idf_measure|
+----------+--------------+-------------------+
|         3|          iraq|0.23809526243476142|
|         3|        showed| 0.1743365558077232|
|         3|  intelligence|0.20782569445751425|
|         3|        strike|0.17411586950893898|
|         3|      official|0.15149485319300557|
|         3|         flows| 0.2774168429760197|
|         3|          main|0.36492623402353547|
|         3|          said|0.06593367258642661|
|         3|         rebel|0.18209445014364567|
|         3|       militia| 0.2252006141545402|
|         3|           oil|0.35763832555989516|
|         3|      pipeline| 0.4720829409342409|
|         3|infrastructure|0.22959926718225876|
|         3|        export|0.23862435123782139|
|         3|   authorities|0.18159366801541998|
|         3|      southern|  0.336553609483104|
|         3|      saturday|0.12197305137253434|
|         3|         halts|0.27365396741

+----------+---------+-------------------+
|doc_Number|     word|     tf_idf_measure|
+----------+---------+-------------------+
|         4|     time|0.10623532598945136|
|         4|       us| 0.1669859687392097|
|         4|  present|0.22209684830286883|
|         4|   record| 0.1232987151692413|
|         4|   months|0.14002501854271598|
|         4|     soar| 0.2306791247647116|
|         4|   prices|0.23156094723382684|
|         4|   barely|0.21935019724396657|
|         4|  wallets| 0.2665151844733088|
|         4|   menace| 0.5747440955975784|
|         4| toppling|0.27964532733021175|
|         4|      oil|0.22253051368171256|
|         4| tearaway| 0.3918885216630942|
|         4|      new| 0.1271397626254836|
|         4|straining| 0.2904044404056468|
|         4|  economy|0.14885602905832815|
|         4|   posing| 0.2589223867776184|
|         4| economic|0.14782686453681568|
|         4|    world|0.09332201126546583|
|         4|elections|0.16009904796740967|
+----------

## 2. SVM objective function
The soft-margin support vector machine classification model minimizes the following objective function (loss).

With $\{\mathbf{x}_i, y_i\}_{i=1}^n$ as the data, $\mathbf{x}_i \in \mathbb{R}^d, y_i \in \{-1, 1\},\mathbf{w}$ as the weight vector, and $b$ as the bias:

$$L(\mathbf{w}, b) = \lambda \|\mathbf{w}\|^2 + n^{-1}\sum_{i=1}^{n} \max\{0, 1 - y_i (\mathbf{w}^\mathsf{T} \mathbf{x}_i + b)\},$$

where $\|\mathbf{w}\|$ is the $L_2$ norm.

The SVM classifier is: $$\widehat{y}_i = \mathrm{sgn}(\mathbf{w}^\mathsf{T} \mathbf{x}_i + b),$$ where $\mathrm{sgn}(z)$ returns the sign of $z$, i.e. $+1$ if positive, $-1$ if negative.

In [19]:
# First, put csvs into Pyspark
svm_ps = spark.read.csv("dataset/data_for_svm.csv", inferSchema=True, header=False)
bias_ps = spark.read.csv("dataset/bias.csv", inferSchema=True, header=False)
w_ps = spark.read.csv("dataset/w.csv", inferSchema=True, header=False)


1. Design the MapReduce functions required to calculate the loss function.

The work comes from trying to calculate this quantity:$$\sum_{i=1}^{n} \max\{0, 1 - y_i (\mathbf{w}^\mathsf{T} \mathbf{x}_i + b)\}$$

In [20]:
# Design the map portion, which calculates the max(0, 1-y_i(w^T x_i +b)) for each row

# First, define global variables outside the function
# Get bias value
bias_val=float(bias_ps.take(1)[0][0])

# Get the w value as a list so we can do w^T * x_i
w_vals=list(w_ps.take(1)[0])

def map_part2(row):
    dict=row.asDict()

    x_col_names=[f'_c{i}' for i in range(64)]
    y_col_name='_c64'

    # Get x values
    x_vals=[]
    for label in x_col_names:
        x_vals.append(dict[label])
 
    # Get the y value
    y_val=dict[y_col_name]

    # Calculate 1-y_i(w^T * x_i + b)
    calculation=1-y_val*(np.dot(np.array(w_vals), np.array(x_vals))+bias_val)

    return ('loss_val', max(0, calculation))
    

25/05/26 19:40:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [21]:
# Design reduce portion, which finds the sum of all max(0, 1-y_i(w^T x_i +b)) values
from operator import add
summed_total=svm_ps.rdd.map(map_part2).reduceByKey(add).collect()[0][1]
summed_total

399889.5049647091

2. Using these functions, create a function loss_SVM(w, b, X, y) to calculate the SVM objective for a given choice of w, b with data stored in X, y.

In [22]:
def loss_SVM(w,b,X,y,lam):
    ###
    # Assuming w, b, X, and y are inputed as PySpark dataframes, and lam is inputed as a float value
    ###
    
    # Get a list of w values
    w_vals=list(w.take(1)[0])

    # Get bias value
    bias_val=float(b.take(1)[0][0])

    # Get n^(-1) value
    n_inv=1/X.count()

    # Join X and y together so we can use our map function from above
    from pyspark.sql.functions import monotonically_increasing_id
    X=X.withColumn('id', monotonically_increasing_id())
    y=y.withColumn('id', monotonically_increasing_id())

    joined_data=X.join(y, on='id').drop('id')

    # Same function as above
    def map_part2(row):
        dict=row.asDict()

        x_col_names=[f'_c{i}' for i in range(64)]
        y_col_name='_c64'

        # Get x values
        x_vals=[]
        for label in x_col_names:
            x_vals.append(dict[label])
    
        # Get the y value
        y_val=dict[y_col_name]

        # Calculate 1-y_i(w^T * x_i + b)
        calculation=1-y_val*(np.dot(np.array(w_vals), np.array(x_vals))+bias_val)

        return ('loss_val', max(0, calculation))
        
    # Again, same reduce function
    summed_total=joined_data.rdd.map(map_part2).reduceByKey(add).collect()[0][1]

    # Return loss objective function using values calculated above
    return lam * np.linalg.norm(w_vals)**2 + n_inv * summed_total

3. You are given the following dataset data_for_svm.csv, where the first 64 columns contain X and the last column contains y. Using the weights and bias provided in w.csv and bias.csv, calculate the objective value.

In [23]:
# Create X and y dfs
X = svm_ps.select(svm_ps.columns[:-1])
y = svm_ps.select(svm_ps.columns[-1])

print('Objective value: ', loss_SVM(w_ps, bias_ps, X, y,1))

Objective value:  1.0029403834857487


4. Design the MapReduce function required to make prediction. Predict for all of the data using the provided weights and bias.

In [24]:
# First, define a mapping function to calculate y_hat for each data point

# Get bias value
bias_val=float(bias_ps.take(1)[0][0])

# Get the w value as a list so we can do w^T * x_i
w_vals=list(w_ps.take(1)[0])

def map_predict(row):
    dict=row.asDict()
    
    x_col_names=[f'_c{i}' for i in range(64)]

    # Get X values
    x_vals=[]
    for label in x_col_names:
        x_vals.append(dict[label])

    
    # Compute inner value in the y_hat score function
    score=np.dot(np.array(w_vals), np.array(x_vals))+bias_val

    # Make the prediction binary based on sign.
    y_hat = 1 if score >= 0 else -1

    return ('Prediction', y_hat)

In [25]:
# Create reduce operation to apply the prediction
preds=X.rdd.map(map_predict)
preds.collect()

[('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Prediction', 1),
 ('Prediction', -1),
 ('Predictio

In [26]:
# For comparison, find the loss of our predictions
preds_df=preds.toDF().select(col('_2').alias('_c64'))
print('Loss of Predictions: ', loss_SVM(w_ps, bias_ps, X, preds_df,1))

Loss of Predictions:  0.9579132416284583
